In [20]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
import re
import openai
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


In [21]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

In [22]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

### CREATE INDEX

In [23]:
docs =  pt_dataset.get_corpus_iter()
docs = list(docs)
count = sum(1 for _ in docs)
docs = docs[:126959]
print("Number of documents:", count)

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 56727.37it/s]

Number of documents: 126958


In [24]:
#Method that removes all special characters from a String, and returns either a String or a list of all words
def clean_text(text, return_as_list = False):
    text = text.replace('\n', ' ')
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) #remove non-alphanumeric characters, except spaces
    if return_as_list:
        word_list = cleaned_text.split()
        return word_list
    else:
        return cleaned_text

In [25]:
import nltk
from nltk.corpus import stopwords
import re

# Ensure NLTK data directory is set correctly
nltk.data.path.append("/usr/local/nltk_data")

# Download 'stopwords' corpus to the specified directory
nltk.download('stopwords', download_dir="/usr/nltk_data")

# Get the set of stopwords for the English language
stop_words = set(stopwords.words('english'))

def remove_stopwords(text, return_as_list = False):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words back into a single string

    if return_as_list:
        return filtered_words
    else:
        return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /usr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

def stem_text(text, return_as_list = False):
    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]
    if return_as_list:
        return stemmed_words
    # Join the stemmed words back into a single string
    else:
        return ' '.join(stemmed_words)

In [27]:
#This is our ngram tokenizer. It takes a string and returns a dict of all ngrams, where each ngram is seperated by $$ so it will be parsed as one token

def tokenize_ngrams_to_dict(text, n1=1, n2=2):
    
    # Tokenize the text into words
    words = text.split(' ')
    words = [word for word in words if len(''.join(format(ord(c), '08b') for c in word)) <= 60]

    # Initialize an empty Counter to hold all n-grams
    all_ngram_counts = Counter()
    
    # Loop through each n from n1 to n2
    for n in range(n1, n2 + 1):
        # Generate n-grams for the current n
        ngrams = ['$$'.join(words[i:i+n]) for i in range(len(words)-n+1)]
        
        # Update the Counter with the current n-grams
        all_ngram_counts.update(ngrams)
    
    return dict(all_ngram_counts)

In [28]:
for doc in docs:
        if 'text' in doc:
            doc['text'] = clean_text(doc['text'])
            doc['text'] = remove_stopwords(doc['text'])
            doc['text'] = stem_text(doc['text'])
            
            doc_1gram = tokenize_ngrams_to_dict(doc['text'], n1=1, n2=2) # Apply n-gram tokenization to the dataset

            doc['toks'] = doc_1gram # create new toks field for tokenfrequency
            del doc['text']  #This will delete the 'text' field from the documents
    
for i, doc in enumerate(docs):
     if i == 3:
           break
     print(doc)

#remove all empty documents
docs = [d for d in docs if any(k != '' for k in d['toks'].keys())]

{'docno': 'O02-2002', 'toks': {'studi': 1, 'word': 7, 'similar': 8, 'use': 3, 'context': 5, 'vector': 3, 'model': 2, 'need': 1, 'measur': 2, 'process': 1, 'natur': 1, 'languag': 1, 'especi': 1, 'general': 1, 'classif': 1, 'exampl': 1, 'base': 3, 'usual': 1, 'two': 1, 'defin': 1, 'accord': 3, 'distanc': 1, 'semant': 6, 'class': 2, 'less': 1, 'consid': 1, 'syntact': 5, 'ie': 2, 'howev': 1, 'real': 1, 'applic': 1, 'requir': 1, 'weight': 1, 'differ': 1, 'mixtur': 1, 'paper': 1, 'propos': 1, 'relat': 1, 'co': 2, 'occurr': 2, 'adopt': 1, 'inform': 1, 'theoret': 1, 'solv': 1, 'problem': 1, 'data': 1, 'spars': 1, 'precis': 1, 'featur': 2, 'deriv': 1, 'pars': 1, 'environ': 1, 'adjust': 1, 'idf': 1, 'invers': 1, 'valu': 2, 'agglom': 1, 'cluster': 1, 'appli': 1, 'group': 2, 'turn': 1, 'togeth': 1, 'studi$$word': 1, 'word$$similar': 4, 'similar$$use': 1, 'use$$context': 1, 'context$$vector': 3, 'vector$$model': 1, 'model$$need': 1, 'need$$measur': 1, 'measur$$word': 1, 'similar$$process': 1, 'proc

In [30]:
# Initialize the IterDictIndexer with pretokenised set to True
iter_indexer = pt.IterDictIndexer("./ngramindex", overwrite=True, meta={'docno': 35}, pretokenised=True, verbose = True, type = pt.index.IndexingType.SINGLEPASS)

# Index our pretokenized documents
index_ref = iter_indexer.index(docs)

index_ngram = pt.IndexFactory.of(index_ref)

#Print some stats about our index
print(index_ngram.getCollectionStatistics())

# Access the MetaIndex and Lexicon
meta = index_ngram.getMetaIndex()
lexicon = index_ngram.getLexicon()


i = 0
for term, le in index_ngram.getLexicon():
    i = i+1
    if i == 5:
        break
    print(term) 
    print(le.getFrequency())

Number of documents: 126824
Number of terms: 1735213
Number of postings: 11740755
Number of fields: 0
Number of tokens: 14688164
Field names: []
Positions:   false

0
9406
0$$0
84
0$$00
11
0$$000
2


### ABBREVIATIONS

In [31]:
client = openai.OpenAI() #connect to OpenAI API

In [32]:
def ask_gpt(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [33]:
querys = pt_dataset.get_topics('query')

In [34]:
def extract_integers_from_qid(df):
    # Convert the qid column to integers
    return df['qid'].astype(int).tolist()

In [35]:
def get_query_by_qid(df, qid):
    # Convert the qid to string and find the corresponding query
    result = df.loc[df['qid'] == str(qid), 'query']
    return result.iloc[0] if not result.empty else None

In [36]:
def set_query_by_qid(df, qid, new_query):
    # Convert the qid to string and update the query
    df.loc[df['qid'] == str(qid), 'query'] = new_query

In [37]:
qid_list = extract_integers_from_qid(querys)

In [38]:
answers = dict()
for i in qid_list:
    determine_abbrevation = f""" 
    You are an scientific expert especially in the domain of Information Retrieval. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, contains an abbrevation then answer with yes or not then answer with no.
    For example given a query 'What is crx' you should answer yes, since cxr is the abbrevation for the medical term
    'chest X-Ray'. However if the given query is 'What is Information Retrieval' you should answer no, since there is no
    abbrevation in the query.

    query: '''{get_query_by_qid(querys, i)}'''
    """
    answer = ask_gpt(prompt=determine_abbrevation) #check answer more carefully perhaps model will return not only {yes,no}
    #print(answer)
    qid = str(i)
    if "yes" in answer.lower().strip():
        answers[qid] = True
    else:
        answers[qid] = False
print(answers)

{'1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': True, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': True, '56': False, '57': False, '58': False, '59': True, '60': False, '61': False, '62': False, '63': False, '64': False, '65': False, '66': False, '67': False, '68': False, '18': False}


In [39]:
for key in answers.keys():
    if bool(answers[key]):
        #find query
        for i in qid_list:
            if str(i) == key:
                query= get_query_by_qid(querys, key)
                print("Old query: ",query)
        #ask gpt to expand query
        expand = f""" 
        You are an scientific expert especially in the domain of Information Retrieval. Your are given a query, which is below
        delimited by triple quotes, which contains an abbrevation. Your task is to identify the abbrevation and write it, then
        concat the original query with the written out abbrevation and return this new query as string only.
        For example given a query 'What is crx' you should detect that the abbrevation is crx, since cxr is the abbrevation for the medical term
        'chest X-Ray', then you should concat the originial query with the abbrevation 'chest X-Ray' resulting in a new query 'What is crx chest x-ray' which
        you should return. Another example, given the query 'Algorithms of nlp' you should detect that the abbrevation is nlp, since nlp is the abbrevation
        for the term 'natural language processing', then you should concat the original query 'Algorithms of nlp' with the abbrevation 'natural language processing'  
        resulting in a new query 'Algorithms of nlp natural language processing' which you should return.
        Please only answer with the new query. So your answer should only include the original query and the detected abbreviated words and no additional information.
        Don't wrap your answer in quotation marks.

        query: '''{query}'''
        """
        new_query = ask_gpt(prompt=expand).lower().strip().replace("'", " ").replace('"', ' ')
        print("New query: ",new_query)
        set_query_by_qid(querys, int(key), new_query) #overwrite old query

Old query:  algorithm acceleration with nvidia cuda
New query:  algorithm acceleration with nvidia cuda compute unified device architecture
Old query:  bm25
New query:  bm25 best match 25
Old query:  what is ahp
New query:  what is ahp analytic hierarchy process


In [40]:
expanded_queries_list = querys.values.tolist()
for i in qid_list:
    

    determine_ngrams = f""" 
    You are an scientific expert in the domain of Information Retrieval and linguistics. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, contains bigrams. This means, you should check for all 
    bigrams in the query, if they are an existing term consisting of multiple words. Then, your answer should be the original query 
    with all the bigrams you found appended in the format word1$$word2. Your answer should only include the query and the bigrams, no additional information.
    This means that when there are no existing bigrams in the query, your answer should just be the original query. You should not wrap your answer in quotation marks.
    For example given a query 'usage of machine learning in image recognition' you should answer
      'usage of machine learning in image recognition machine$$learning image$$recognition'.

    query: '''{get_query_by_qid(querys, i)}'''
    """
    answer = ask_gpt(prompt=determine_ngrams)
    set_query_by_qid(querys, i, answer)

expanded_queries_list = querys.values.tolist()
print(expanded_queries_list)

[['1', 'retrieval system improving effectiveness retrieval$$system'], ['2', 'machine learning language identification machine$$learning language$$identification'], ['3', 'social media detect self harm social$$media self$$harm'], ['4', 'stemming for arabic languages arabic$$languages'], ['5', 'audio based animal recognition audio$$based animal$$recognition'], ['6', 'comparison different retrieval models retrieval$$models'], ['7', 'cache architecture cache$$architecture'], ['8', 'document scoping formula document$$scoping'], ['9', 'pseudo relevance feedback pseudo$$relevance relevance$$feedback'], ['10', 'how to represent natural conversations in word nets natural$$conversations word$$nets'], ['11', 'algorithm acceleration with nvidia cuda compute unified device architecture nvidia$$cuda compute$$unified unified$$device device$$architecture'], ['12', 'mention of algorithm'], ['13', 'at least three authors at$$least three$$authors'], ['14', 'german domain german$$domain'], ['15', 'mention

In [41]:
data_expd = [(sublist[0], sublist[1]) for sublist in expanded_queries_list]
print(data_expd)

[('1', 'retrieval system improving effectiveness retrieval$$system'), ('2', 'machine learning language identification machine$$learning language$$identification'), ('3', 'social media detect self harm social$$media self$$harm'), ('4', 'stemming for arabic languages arabic$$languages'), ('5', 'audio based animal recognition audio$$based animal$$recognition'), ('6', 'comparison different retrieval models retrieval$$models'), ('7', 'cache architecture cache$$architecture'), ('8', 'document scoping formula document$$scoping'), ('9', 'pseudo relevance feedback pseudo$$relevance relevance$$feedback'), ('10', 'how to represent natural conversations in word nets natural$$conversations word$$nets'), ('11', 'algorithm acceleration with nvidia cuda compute unified device architecture nvidia$$cuda compute$$unified unified$$device device$$architecture'), ('12', 'mention of algorithm'), ('13', 'at least three authors at$$least three$$authors'), ('14', 'german domain german$$domain'), ('15', 'mention

In [43]:
#expanded_queries = [['1', 'retrieval system improving effectiveness', 'retrieval system improving effectiveness retrieval$$system'], ['2', 'machine learning language identification', 'machine learning language identification machine$$learning language$$identification'], ['3', 'social media detect self harm', 'social media detect self harm social$$media self$$harm'], ['4', 'stemming for arabic languages', 'stemming for arabic languages arabic$$languages'], ['5', 'audio based animal recognition', 'audio based animal recognition audio$$based animal$$recognition'], ['6', 'comparison different retrieval models', 'comparison different retrieval models retrieval$$models'], ['7', 'cache architecture', 'cache architecture'], ['8', 'document scoping formula', 'document scoping formula document$$scoping'], ['9', 'pseudo relevance feedback', 'pseudo relevance feedback pseudo$$relevance$$feedback pseudo$$relevance relevance$$feedback'], ['10', 'how to represent natural conversations in word nets', 'how to represent natural conversations in word nets natural$$conversations word$$nets'], ['11', 'algorithm acceleration with nvidia cuda', 'algorithm acceleration with nvidia cuda nvidia$$cuda'], ['12', 'mention of algorithm', 'mention of algorithm'], ['13', 'at least three authors', 'at least three authors three$$authors'], ['14', 'german domain', 'german domain german$$domain'], ['15', 'mention of open source', 'mention of open source open$$source'], ['16', 'inclusion of text mining', 'inclusion of text mining text$$mining'], ['17', 'the ethics of artificial intelligence', 'the ethics of artificial intelligence artificial$$intelligence'], ['19', 'machine learning for more relevant results', 'machine learning for more relevant results machine$$learning relevant$$results'], ['20', 'crawling websites using machine learning', 'crawling websites using machine learning machine$$learning'], ['21', 'recommenders influence on users', 'recommenders influence on users'], ['22', 'search engine caching effects', 'search engine caching effects search$$engine caching$$effects'], ['23', 'consumer product reviews', 'consumer product reviews consumer$$product product$$reviews'], ['24', 'limitations machine learning', 'limitations machine learning machine$$learning'], ['25', 'medicine related research', 'medicine related research medicine$$related related$$research'], ['26', 'natural language processing', 'natural language processing natural$$language$$processing natural$$language'], ['27', 'graph based ranking', 'graph based ranking graph$$based based$$ranking'], ['28', 'medical studies that use information retrieval', 'medical studies that use information retrieval information$$retrieval'], ['29', 'information retrieval on different language sources', 'information retrieval on different language sources information$$retrieval language$$sources'], ['30', 'papers that compare multiple information retrieval methods', 'papers that compare multiple information retrieval methods information$$retrieval'], ['31', 'risks of information retrieval in social media', 'risks of information retrieval in social media information$$retrieval social$$media'], ['32', 'actual experiments that strengthen theoretical knowledge', 'actual experiments that strengthen theoretical knowledge theoretical$$knowledge'], ['33', 'fake news detection', 'fake news detection fake$$news news$$detection'], ['34', 'multimedia retrieval', 'multimedia retrieval multimedia$$retrieval'], ['35', 'processing natural language for information retrieval', 'processing natural language for information retrieval natural$$language information$$retrieval'], ['36', 'recommendation systems', 'recommendation systems recommendation$$systems'], ['37', 'personalised search in e commerce', 'personalised search in e commerce personalised$$search e$$commerce'], ['38', 'sentiment analysis', 'sentiment analysis sentiment$$analysis'], ['39', 'informational retrieval using neural networks', 'informational retrieval using neural networks informational$$retrieval neural$$networks'], ['40', 'query log analysis', 'query log analysis query$$log log$$analysis'], ['41', 'entity recognition', 'entity recognition entity$$recognition'], ['42', 'relevance assessments', 'relevance assessments relevance$$assessments'], ['43', 'deep neural networks', 'deep neural networks deep$$neural neural$$networks'], ['44', 'information retrieval', 'information retrieval information$$retrieval'], ['45', 'analysis for android apps', 'analysis for android apps android$$apps'], ['46', 'the university of amsterdam', 'the university of amsterdam university$$of of$$amsterdam'], ['47', 'neural ranking for ecommerce product search', 'neural ranking for ecommerce product search neural$$ranking product$$search'], ['48', 'web pages evolution', 'web pages evolution web$$pages'], ['49', 'exhaustivity of index', 'exhaustivity of index'], ['50', 'query optimization', 'query optimization query$$optimization'], ['51', 'cosine similarity vector', 'cosine similarity vector cosine$$similarity'], ['52', 'reverse indexing', 'reverse indexing reverse$$indexing'], ['53', 'index compression techniques', 'index compression techniques index$$compression compression$$techniques'], ['54', 'search engine optimization with query logs', 'search engine optimization with query logs search$$engine engine$$optimization query$$logs'], ['55', 'bm25', 'bm25'], ['56', 'what makes natural language processing natural', 'what makes natural language processing natural natural$$language$$processing natural$$language'], ['57', 'principle of a information retrieval indexing', 'principle of a information retrieval indexing information$$retrieval'], ['58', 'architecture of web search engine', 'architecture of web search engine web$$search search$$engine'], ['59', 'what is ahp', 'what is ahp'], ['60', 'what is information retrieval', 'what is information retrieval information$$retrieval'], ['61', 'efficient retrieval algorithms', 'efficient retrieval algorithms retrieval$$algorithms'], ['62', 'how to avoid spam results', 'how to avoid spam results spam$$results'], ['63', 'information retrieval with algorithms', 'information retrieval with algorithms information$$retrieval'], ['64', 'misspellings in queries', 'misspellings in queries'], ['65', 'information in different language', 'information in different language different$$language'], ['66', 'abbreviations in queries', 'abbreviations in queries'], ['67', 'lemmatization algorithms', 'lemmatization algorithms'], ['68', 'filter ad rich documents', 'filter ad rich documents ad$$rich'], ['18', 'advancements in information retrieval', 'advancements in information retrieval information$$retrieval']]
#print(expanded_queries)

#data_expd = [(sublist[0], sublist[1]) for sublist in expanded_queries] # Extract qid and query from each sublist


df_expd = pd.DataFrame(data_expd, columns=['qid', 'query']) # Create pandas DataFrame

In [44]:
df_expd[60:70]

,qid,query
60,62,how to avoid spam results spam$$results
61,63,information retrieval with algorithms informat...
62,64,misspellings in queries
63,65,information in different language different$$l...
64,66,abbreviations in queries
65,67,lemmatization algorithms
66,68,filter ad rich documents ad$$rich
67,18,advancements in information retrieval informat...


### Clean the text

In [45]:
def clean_text_with_dollar_signs(text, return_as_list=False, keep_dollar_signs=False):
    text = text.replace('\n', ' ')
    
    if keep_dollar_signs:      
        text = text.replace('$$', 'DOUBLEDOLLARNGRAMS')                 # Replace double dollar signs with a unique placeholder
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)             # Remove all non-alphanumeric characters except spaces    
        cleaned_text = cleaned_text.replace('DOUBLEDOLLARNGRAMS', '$$') # Replace placeholder back to double dollar signs
    else:
        
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)             # Remove all non-alphanumeric characters except spaces
    
    if return_as_list:
        word_list = cleaned_text.split()
        return word_list
    else:
        return cleaned_text

In [46]:
transf_clean_text = pt.rewrite.tokenise(lambda query: clean_text_with_dollar_signs(query, return_as_list=True, keep_dollar_signs=True))

### remvoe stopwords

In [47]:
# Ensure NLTK data directory is set correctly
nltk.data.path.append("/usr/local/nltk_data")

# Download 'stopwords' corpus to the specified directory
nltk.download('stopwords', download_dir="/usr/nltk_data")

# Get the set of stopwords for the English language
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
def remove_stopwords_with_dollar_signs(text, return_as_list=False):
    words = text.split()
    filtered_words = []
    
    for word in words:
        if '$$' in word:
            parts = word.split('$$')
            skip_word = False
            for part in parts:
                if part.lower() in stop_words:
                    skip_word = True
                    break  # If any part is a stopword, skip the entire word
            if not skip_word:
                filtered_words.append(word)
        else:
            if word.lower() not in stop_words:
                filtered_words.append(word)
    
    if return_as_list:
        return filtered_words
    else:
        return ' '.join(filtered_words)

In [49]:
transf_remove_stopwords = pt.rewrite.tokenise(lambda query: remove_stopwords_with_dollar_signs(query, return_as_list=True))

### stemming

In [50]:
stemmer = SnowballStemmer('english')

In [51]:
def stem_text_with_dollar_signs(text, return_as_list=False):
    words = text.split()

    stemmed_words = []
    for word in words:
        if '$$' in word:
            parts = word.split('$$')
            stemmed_parts = [stemmer.stem(part) for part in parts]
            stemmed_word = '$$'.join(stemmed_parts)
        else:
            stemmed_word = stemmer.stem(word)   # Stem the word normally
        
        stemmed_words.append(stemmed_word)

    if return_as_list:
        return stemmed_words
    else:
        return ' '.join(stemmed_words)

In [52]:
transf_stem_text = pt.rewrite.tokenise(lambda query: stem_text_with_dollar_signs(query, return_as_list=True))

### Pipeline

In [53]:
index_factory = pt.IndexFactory.of("./ngramindex/data.properties")

In [54]:
# This transformer will do the retrieval using bm25, and explicitly not apply any stemming and stopword removal
bm25 = pt.BatchRetrieve(index_factory, wmodel="BM25", verbose = True, properties={"termpipelines" : ""}, controls={"bm25.b": 0.2})#, "bm25.k_1": 0.1})

# This is our retrieval pipeline
retr_pipeline = transf_clean_text >> transf_remove_stopwords >> transf_stem_text >> bm25

In [55]:
run = retr_pipeline(df_expd) #queries

BR(BM25):   0%|          | 0/68 [00:00<?, ?q/s]

BR(BM25): 100%|██████████| 68/68 [00:02<00:00, 24.28q/s]


In [57]:
persist_and_normalize_run(run, system_name='ngrams', default_output='./runs/fullrun')

#import os
#os.rename('../runs/run.txt', '../runs/runngram.txt')

The run file is normalized outside the TIRA sandbox, I will store it at "./runs/fullrun".
Done. run file is stored under "./runs/fullrun/run.txt".


: 